In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [5]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6144, 28, 28, 1)


In [6]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6144, 10)


In [7]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(6144, 26)


In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_letter_train.shape)
print(x_letter_val.shape)

(4915, 28, 28, 1)
(1229, 28, 28, 1)
(4915, 10)
(1229, 10)
(4915, 26)
(1229, 26)


In [9]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(model_input, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=1)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    
    return model

In [10]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48],
                   'myDenseNet':[8, 12, 16, 32]}

base_growth_rate = 32

model_input = Input(shape=(28,28,1))
classes = 10

model = DenseNet(model_input, classes, 'myDenseNet')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 28, 28, 64)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

In [11]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

#es = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp])

Train on 4915 samples, validate on 1229 samples
Epoch 1/100
4915/4915 [==============================] - 54s 11ms/step - loss: 1.9892 - acc: 0.3070 - val_loss: 8.6995 - val_acc: 0.1155
Epoch 2/100
4915/4915 [==============================] - 28s 6ms/step - loss: 1.1622 - acc: 0.6004 - val_loss: 2.7447 - val_acc: 0.3531
Epoch 3/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.8627 - acc: 0.7076 - val_loss: 1.9554 - val_acc: 0.5378
Epoch 4/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.6482 - acc: 0.7827 - val_loss: 1.4524 - val_acc: 0.6330
Epoch 5/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.4976 - acc: 0.8307 - val_loss: 1.2504 - val_acc: 0.6688
Epoch 6/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.4102 - acc: 0.8564 - val_loss: 1.4170 - val_acc: 0.6640
Epoch 7/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.3604 - acc: 0.8730 - val_loss: 2.1664 - val_acc: 0.5989

4915/4915 [==============================] - 28s 6ms/step - loss: 0.0066 - acc: 0.9982 - val_loss: 0.3212 - val_acc: 0.9186
Epoch 61/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0223 - acc: 0.9923 - val_loss: 0.8910 - val_acc: 0.8421
Epoch 62/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0239 - acc: 0.9927 - val_loss: 1.0166 - val_acc: 0.8234
Epoch 63/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0385 - acc: 0.9892 - val_loss: 0.3508 - val_acc: 0.9203
Epoch 64/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0614 - acc: 0.9841 - val_loss: 1.1972 - val_acc: 0.7852
Epoch 65/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0475 - acc: 0.9862 - val_loss: 1.0736 - val_acc: 0.8218
Epoch 66/100
4915/4915 [==============================] - 28s 6ms/step - loss: 0.0434 - acc: 0.9851 - val_loss: 0.4528 - val_acc: 0.9170
Epoch 67/100
4915/4915 [==============================

In [15]:
from tensorflow.keras.models import load_model
best_model = load_model('models/myDenseNet_submit.h5')

In [16]:
x_test = test[:,1:]
x_test = np.reshape(x_test, (-1, 28, 28, 1))
print(x_test.shape)

(20480, 28, 28, 1)


In [17]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = np.argmax(best_model.predict(x_test), axis=1)
submission.to_csv('data/submission_mydensenet(0822).csv', index=False)